In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import math

In [15]:
dec_2016 = pd.read_csv('data/raw_data/Video_Games_Sales_as_at_22_Dec_2016.csv')
jan_2017 = pd.read_csv('data/raw_data/Video_Game_Sales_as_of_Jan_2017.csv')

In [16]:
''' Read all the csv's into a list. '''
year_csv = {}
for csv in glob.iglob('data/time_series_data/*.csv'):
    thing = csv.split("/")
    year = thing[2].split(".")[0]
    year_csv[year] = csv

In [17]:
def thingaling(year_csv):
    for title in year_csv:
        if title in pd.Series(dec_2016['Name']).tolist():
            print '2016'
        elif title in pd.Series(jan_2017['Name']).tolist():
            print '2017'

In [37]:
''' Returns a mapping of each year to the number of sales of games of the given feature in the given region. '''
def calc_sales_for_feature(year_csv, feature, feature_value, sales_region):
    sales = {}
    
    for k in range(len(dec_2016)):
        feat = dec_2016.iloc[k][feature]
        year = dec_2016.iloc[k]['Year_of_Release']
        if math.isnan(year):
            continue
        
        if year not in sales.keys():
            sales[int(year)] = 0.
        if feat == feature_value:
            sales[year] += dec_2016.iloc[k][sales_region]

    for k in range(len(jan_2017)):
        feat = jan_2017.iloc[k][feature]
        year = jan_2017.iloc[k]['Year_of_Release']
        if math.isnan(year):
            continue 
        
        if year not in sales.keys():
            sales[int(year)] = 0.
        if feat == feature_value:
            sales[year] += jan_2017.iloc[k][sales_region]
    return sales

In [38]:
#blah = pd.read_csv(year_csv['1995'])
#for x in range(len(blah)):
#    print blah.iloc[x]['Name']

print calc_sales_for_feature(year_csv, 'Genre', 'Platform', 'Global_Sales')

{1976: 0.0, 1977: 0.0, 1978: 0.0, 1979: 0.0, 1980: 0.0, 1981: 13.860000000000001, 1982: 10.060000000000002, 1983: 13.859999999999998, 1984: 1.38, 1985: 86.34, 1986: 18.78, 1987: 3.4800000000000004, 1988: 55.46000000000001, 1989: 41.32000000000001, 1990: 45.94, 1991: 15.28, 1992: 26.839999999999996, 1993: 37.34, 1994: 57.48, 1995: 33.38, 1996: 56.46, 1997: 44.73, 1998: 59.75999999999999, 1999: 41.64, 2000: 32.11999999999999, 2001: 78.56000000000002, 2002: 92.22999999999999, 2003: 85.78, 2004: 94.48000000000006, 2005: 46.97000000000005, 2006: 99.31999999999995, 2007: 70.22999999999995, 2008: 70.33000000000004, 2009: 81.25999999999999, 2010: 62.11, 2011: 55.52, 2012: 36.99999999999999, 2013: 49.59, 2014: 17.78, 2015: 12.329999999999995, 2016: 8.599999999999998, 2017: 0.0, 2020: 0.0}
